# Fake News Detection 🕵️🗞️

## Get preprocessed data

In [1]:
!wget -O train.csv 'https://drive.google.com/u/0/uc?id=1pqhbkJl1zV8ZnB4MhKuRtGPtjmYh4WxA&export=download'

--2023-03-26 03:24:29--  https://drive.google.com/u/0/uc?id=1pqhbkJl1zV8ZnB4MhKuRtGPtjmYh4WxA&export=download
Resolving drive.google.com (drive.google.com)... 108.177.119.102, 108.177.119.100, 108.177.119.101, ...
Connecting to drive.google.com (drive.google.com)|108.177.119.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1pqhbkJl1zV8ZnB4MhKuRtGPtjmYh4WxA&export=download [following]
--2023-03-26 03:24:30--  https://drive.google.com/uc?id=1pqhbkJl1zV8ZnB4MhKuRtGPtjmYh4WxA&export=download
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jkcc2g8v7m7l2gbkbu52iaeu4mlme7ho/1679801025000/04497796494746103904/*/1pqhbkJl1zV8ZnB4MhKuRtGPtjmYh4WxA?e=download&uuid=96bd9cce-c957-482f-9d67-de9d0a0a9245 [following]
--2023-03-26 03:24:33--  https://doc-04-4c-docs.googleuserconten

In [2]:
!wget -O test.csv 'https://drive.google.com/u/0/uc?id=1KLZE-XyNXfVQTYGJHrFB-5Vqsukt8Cms&export=download'

--2023-03-26 03:24:35--  https://drive.google.com/u/0/uc?id=1KLZE-XyNXfVQTYGJHrFB-5Vqsukt8Cms&export=download
Resolving drive.google.com (drive.google.com)... 108.177.119.102, 108.177.119.100, 108.177.119.101, ...
Connecting to drive.google.com (drive.google.com)|108.177.119.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1KLZE-XyNXfVQTYGJHrFB-5Vqsukt8Cms&export=download [following]
--2023-03-26 03:24:35--  https://drive.google.com/uc?id=1KLZE-XyNXfVQTYGJHrFB-5Vqsukt8Cms&export=download
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gggrfsd3qb75q05p6jlj96pf0msb362q/1679801025000/04497796494746103904/*/1KLZE-XyNXfVQTYGJHrFB-5Vqsukt8Cms?e=download&uuid=557d5c8e-2056-416e-b7f7-2e35b96a5b95 [following]
--2023-03-26 03:24:47--  https://doc-0k-4c-docs.googleuserconten

In [3]:
import pandas as pd
train_df = pd.read_csv('train.csv').drop('Unnamed: 0', axis=1)
test_df = pd.read_csv('test.csv').drop('Unnamed: 0', axis=1)
train_df.head()

,Text,Label
0,"Like a little crew of bandits, the first few...",0
1,"On Thursday’s Breitbart News Daily, SiriusXM h...",0
2,Home › POLITICS › IF HILLARY CLINTON IS CHARGE...,1
3,The New York Times’s Moscow bureau was the tar...,0
4,An event organised by Gays Against Sharia to m...,0


In [4]:
test_df.head()

,Text,Label
0,By Bev Harris A real-time demo of the most dev...,1
1,"\nBy Peter Koenig , November \nThe elections...",1
2,Just days after a national campaign in which h...,0
3,House Intelligence Committee chair Rep. Devin ...,0
4,Hillary’s Secret Is Out With What Camera Caugh...,1


In [5]:
train_df.shape

(16640, 2)

In [6]:
test_df.shape

(4160, 2)

In [7]:
#Setup train_sentences
train_sentences_numpy = train_df['Text'].to_numpy()
train_labels = train_df['Label'].to_numpy()

#Setup test sentences
test_sentences_numpy = test_df['Text'].to_numpy()
test_labels = test_df['Label'].to_numpy()

In [8]:
train_sentences_numpy[0]

'Like a little crew of   bandits, the first few raccoon faces poked out of the underbrush on a recent evening in Central Park, their masked, button eyes glowing with each flash of a smartphone camera. Undaunted, several of the animals scuttled onto the walkway toward the picture takers. They were followed by more raccoons  —   and then still more. All told,  raccoons clustered on a path near the southeastern edge of the park, where it meets th Street in Manhattan. They strolled around while a gaggle of onlookers tried to pose for selfies with them. Some people fed the animals soft pretzels from a pushcart by hand, as the raccoons stood on their hind legs to wrest morsels from their admirers’ fingertips. “Is this normal?” Karen Newis, , a visitor from Greensboro, N. C. asked her sister Stefanie Price, who lives in Manhattan, as they passed the raccoons. One of the animals gingerly tiptoed toward a baby in a stroller, sniffing. “No, it’s not normal,” Ms. Price, , said. “They’re so cute, 

## Modelling Experiments

Here's all the models, I'm going to create and run

- Model 0 : a baseline naive bayes model
- Model 1 : A Conv1D layer with a global maxpooling layer
- Model 2 : A LSTM
- Model 3 : A GRU
- Model 4 : A Bidirectional RNN
- Model 5 : Using a universal sentence encoder

All the following models will be uploaded to tensorboard for futher evaluation

### Model 0

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

#Preprocess
train_sentences_pd = train_df['Text'].fillna('')
test_sentences_pd = test_df['Text'].fillna('')

In [10]:
#Create model0
model0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ('clf', MultinomialNB())
])

#Fit model0
model0.fit(train_sentences_pd, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [11]:
results_model0 = model0.score(test_sentences_pd, test_labels)
results_model0

0.8521634615384616

### Model 1

In [12]:
import tensorflow as tf

#Create vectorization layer
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=1000)
text_vectorizer.adapt(train_sentences_pd)

In [13]:
#Get vocabulary
vocabulary = text_vectorizer.get_vocabulary()

#Setup embedding layer
embedding = tf.keras.layers.Embedding(input_dim=len(vocabulary),
                                      output_dim=128)

In [14]:
vocabulary[:10]

['', '[UNK]', 'the', 'to', 'of', 'and', 'a', 'in', 'that', 'is']

In [15]:
text_vectorizer(train_sentences_numpy[0])

<tf.Tensor: shape=(1000,), dtype=int64, numpy=
array([    63,      6,    256,   3064,      4,  27588,      2,     94,
          215,  42246,   2132,  17261,     58,      4,      2,  61958,
           11,      6,    279,   1188,      7,    637,   1179,     38,
        11761,   5818,   1429,  11908,     12,    294,   6368,      4,
            6,   5355,   2480,  31530,    252,      4,      2,   2562,
        34944,   1767,      2,  32964,    634,      2,   1497,  22367,
           31,     46,   1247,     18,     47,  24477,     36,      5,
          113,    149,     47,     53,    148,  24477,  18105,     11,
            6,   1624,    581,      2,  10975,   2724,      4,      2,
         1179,    107,     14,   6068,    447,    453,      7,   1811,
           31,  20757,    177,    114,      6,  19570,      4,  16419,
          720,      3,   4712,     10,  10388,     12,     76,     70,
           51,   1583,      2,   2562,   4473,  45751,     25,      6,
       170419,     18,    735,

In [16]:
#Import dependencies
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Conv1D, GlobalMaxPool1D

#Create inputs and outputs
inputs = Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = Conv1D(64, 3)(x)
x = GlobalMaxPool1D()(x)
outputs = Dense(1, activation='sigmoid')(x)

#Create the model
model1 = tf.keras.Model(inputs, outputs)

In [17]:
#Compile the model
model1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

#Create tensorboard callback
model1_tensorboard = tf.keras.callbacks.TensorBoard(log_dir='/content/Models/Model1')

In [18]:
history_model1 = model1.fit(train_sentences_pd,
                            train_labels,
                            epochs=2,
                            validation_data=(test_sentences_pd, test_labels),
                            callbacks=[model1_tensorboard])

Epoch 1/2
520/520 [==============================] - 97s 169ms/step - loss: 0.1797 - accuracy: 0.9314 - val_loss: 0.0591 - val_accuracy: 0.9781
Epoch 2/2
520/520 [==============================] - 67s 130ms/step - loss: 0.0289 - accuracy: 0.9908 - val_loss: 0.0360 - val_accuracy: 0.9856


In [19]:
results_model1 = model1.evaluate(test_sentences_pd, test_labels)
results_model1

130/130 [==============================] - 1s 9ms/step - loss: 0.0360 - accuracy: 0.9856


[0.036042891442775726, 0.9855769276618958]

In [20]:
#Create a function to make custom predictions
def predict_with_model(model):
  text = input('Enter the news here: ')
  results = model.predict([text])
  if results < 0.5:
    print('This news is reliable')
  else:
    print('This news is unreliable')


```
1: unreliable
0: reliable
```

### Model 2

In [21]:
#Import dependencies
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

#Create inputs and outputs
inputs = Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.LSTM(40)(x)
outputs = Dense(1, activation='sigmoid')(x)

#Create the model and compile
model2 = tf.keras.Model(inputs, outputs)
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

#Create tensorboard callback and fit model
model2_tensorboard = tf.keras.callbacks.TensorBoard(log_dir='/content/Models/Model2')
history_model2 = model2.fit(train_sentences_pd,
                            train_labels,
                            epochs=2,
                            validation_data=(test_sentences_pd, test_labels),
                            callbacks=[model2_tensorboard])

Epoch 1/2
520/520 [==============================] - 90s 166ms/step - loss: 0.6588 - accuracy: 0.6043 - val_loss: 0.6127 - val_accuracy: 0.6243
Epoch 2/2
520/520 [==============================] - 69s 132ms/step - loss: 0.5154 - accuracy: 0.7496 - val_loss: 0.3489 - val_accuracy: 0.8805


In [22]:
resuts_model2 = model2.evaluate(test_sentences_pd, test_labels)

130/130 [==============================] - 2s 19ms/step - loss: 0.3489 - accuracy: 0.8805


### Model 3 

In [23]:
#Import dependencies
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

#Create inputs and outputs
inputs = Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GRU(40)(x)
outputs = Dense(1, activation='sigmoid')(x)

#Create the model and compile
model3 = tf.keras.Model(inputs, outputs)
model3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

#Create tensorboard callback and fit model
model3_tensorboard = tf.keras.callbacks.TensorBoard(log_dir='/content/Models/Model3')
history_model3 = model3.fit(train_sentences_pd,
                            train_labels,
                            epochs=2,
                            validation_data=(test_sentences_pd, test_labels),
                            callbacks=[model3_tensorboard])

Epoch 1/2
520/520 [==============================] - 94s 176ms/step - loss: 0.6382 - accuracy: 0.6603 - val_loss: 0.5489 - val_accuracy: 0.6971
Epoch 2/2
520/520 [==============================] - 68s 132ms/step - loss: 0.3024 - accuracy: 0.8951 - val_loss: 0.2436 - val_accuracy: 0.9202


In [24]:
results_model3 = model3.evaluate(test_sentences_pd, test_labels)

130/130 [==============================] - 2s 17ms/step - loss: 0.2436 - accuracy: 0.9202


### Model 4

In [25]:
#Import dependencies
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

#Create inputs and outputs
inputs = Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Bidirectional(LSTM(40))(x)
outputs = Dense(1, activation='sigmoid')(x)

#Create the model and compile
model4 = tf.keras.Model(inputs, outputs)
model4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

#Create tensorboard callback and fit model
model4_tensorboard = tf.keras.callbacks.TensorBoard(log_dir='/content/Models/Model4')
history_model4 = model4.fit(train_sentences_pd,
                            train_labels,
                            epochs=2,
                            validation_data=(test_sentences_pd, test_labels),
                            callbacks=[model4_tensorboard])

Epoch 1/2
520/520 [==============================] - 97s 179ms/step - loss: 0.2700 - accuracy: 0.8954 - val_loss: 0.1829 - val_accuracy: 0.9349
Epoch 2/2
520/520 [==============================] - 78s 151ms/step - loss: 0.0947 - accuracy: 0.9683 - val_loss: 0.1288 - val_accuracy: 0.9519


In [26]:
results_model4 = model4.evaluate(test_sentences_pd, test_labels)

130/130 [==============================] - 5s 35ms/step - loss: 0.1288 - accuracy: 0.9519


### Model 5

In [31]:
#Import dependencies
import tensorflow_hub as hub

In [34]:
sentence_encoder = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4', trainable=False, input_shape=[], dtype=tf.string)

In [43]:
#Create our model
model5 = tf.keras.Sequential([
    sentence_encoder,
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

#Compile model
model5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

In [44]:
#create callback
model5_tensorboard = tf.keras.callbacks.TensorBoard(log_dir='/content/Models/Model5')

#Fit model
history_model5 = model5.fit(train_sentences_pd,
                            train_labels,
                            epochs=2,
                            validation_data=(test_sentences_pd, test_labels),
                            callbacks=[model5_tensorboard])

Epoch 1/2
520/520 [==============================] - 90s 163ms/step - loss: 0.3810 - accuracy: 0.8392 - val_loss: 0.3099 - val_accuracy: 0.8675
Epoch 2/2
520/520 [==============================] - 87s 167ms/step - loss: 0.2813 - accuracy: 0.8822 - val_loss: 0.2793 - val_accuracy: 0.8813


In [45]:
results_model5 = model5.evaluate(test_sentences_pd, test_labels)

130/130 [==============================] - 18s 137ms/step - loss: 0.2793 - accuracy: 0.8813


## Evaluating models

In [47]:
#Let's just do a quick check of each model's results
print(f'Model 0: {results_model0}')
print(f'Model 1: {results_model1[1]}')
print(f'Model 2: {resuts_model2[1]}')
print(f'Model 3: {results_model3[1]}')
print(f'Model 4: {results_model4[1]}')
print(f'Model 5: {results_model5[1]}')

Model 0: 0.8521634615384616
Model 1: 0.9855769276618958
Model 2: 0.8805288672447205
Model 3: 0.920192301273346
Model 4: 0.9519230723381042
Model 5: 0.8812500238418579


In [49]:
#Lets upload to tensorboard
!tensorboard dev upload --logdir '/content/Models' --one_shot

2023-03-26 04:08:01.035138: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-26 04:08:01.035239: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-26 04:08:01.035259: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

/content/Models

This TensorBoard 